#1. 파일 읽기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

filepath='/content/drive/MyDrive/4. 머신러닝/1. 분석 기초/2. 데이터 분석을 위한 기초 통계 수학/3. 파이프라인/data/titanic.csv'
df_titanic = pd.read_csv(filepath)
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#2. 데이터 전처리

##2.1 데이터 추출(Quiz)

> 데이터에는 객실등급, 성별, 나이를 label에는 생존 정보를 저장하시오

In [3]:
df_data = df_titanic[['Pclass', 'Sex', 'Age']]
df_target = df_titanic['Survived']
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  891 non-null    int64  
 1   Sex     891 non-null    object 
 2   Age     714 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 21.0+ KB


> 생존에 영향을 주는 데이터는 위의 세가지

##2.2 데이터 변환(Quiz)

> 문자열은 학습을 시킬 수 없음으로 수치데이터로 변환해야 한다.

In [ ]:
# 기존에 했던 정보
from sklearn import preprocessing

features = ['Cabin', 'Sex', 'Embarked']

for feature in features:
  # LabelEncode 로드
  le = preprocessing.LabelEncoder()
  # feature의 크기 설정
  le = le.fit(df_titanic[feature])
  # 데이터 변환
  df_titanic[feature] = le.transform(df_titanic[feature])

In [ ]:
df_data['Sex'] = df_data['Sex'].map({'female':0, 'male':1})
df_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Pclass,Sex,Age
0,3,1,22.0
1,1,0,38.0
2,3,0,26.0
3,1,0,35.0
4,3,1,35.0


> 기존에 문자열로 되어 있던 성별을 0, 1로 변환시켰다.

##2.3 결측치 변환(Quiz)

> 현재 나이에 결측치가 나타나 평균나이로 변환시킬 것이다.

In [ ]:
df_data['Age'].fillna(df_data['Age'].mean(), inplace=True)
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  891 non-null    int64  
 1   Sex     891 non-null    int64  
 2   Age     891 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 21.0 KB


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


#3. 모델 만들기

##3.1 정규화

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
standard_scaled = scaler.fit_transform(df_data)

###3.2 데이터 추출

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(standard_scaled, df_target, test_size=0.3, random_state=62)

##3.3모델 생성

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

##3.4 성능평가

In [ ]:
lr.score(X_test, y_test)

0.7947761194029851

##3.5 회귀 계수
> 회귀 계수가 음수가 되면 데이터의 수치가 커질 수록 결과가 작은 수일 확률이 높다. 양수가 되면 데이터의 수차가 커질 수록 결과가 큰 수일 확률이 높다.

In [ ]:
lr.

AttributeError: ignored

In [ ]:
lr.coef_

array([[-0.96190556, -1.20405933, -0.38680358]])

In [ ]:
result = pd.DataFrame(lr.coef_, columns=list(scaler.get_feature_names_out()))
result

,Pclass,Sex,Age
0,-0.961906,-1.204059,-0.386804


> 회귀 계수가 음수가 되면 수치가 커질 수록 작은 수일 확률이 높다. 양수이면 수치가 커질수록 큰 수일 확률이 높다. 
* 첫 번째 데이터는 자리 등급으로써 음수임으로 커질수록 즉, C등급 자리일수록 작은 수일 확률 즉, 0인 죽을 확률이 높다.
* 두 번째 데이터는 성별로써 음수임으로 커질수록 즉, 1인 남자일 수록 작은 수일 확률 즉, 0인 죽을 확률이 높다.
* 세 번째 데이터는 나이로써 음수임으로 커질수록 즉, 나이가 많을수록 작은 수일 확률 즉, 0인 죽을 확률이 높다.